# Project 2
For project 2, we built an ETL pipeline to create a database containing data on Phish live shows from 1993-2023. First, we scrapped the website phish.net for setlists and the Wikipedia page for Phish concert tours and festivals for attendance and box office data. We then transformed the extracted data, reformatting columns, value formatting, and dropping rows that lacked useful data. We stored all of the results for each year in a variable, which were in turn each stored in a list. To load our data, we converted each variable to a DataFrame and then wrote the DataFrames to CSV files. Finally, we loaded the data from the CSV files directly to SQLite and PostgreSQL. There are many interesting questions that can be explored when analyzing the database we have prepared. For example, we could look at how City and Year affect Attendance and Attendance/Capacity. We could count the recurrence of previous Cities and weigh Attendance and Gross to create a predictive model to determine the likeliest cities to be announced for future show dates. 

In [1]:
from datetime import datetime
import pandas as pd
import requests
from bs4 import BeautifulSoup
import os
# import shutil
# from pathlib import Path
import pycountry

## Part 1: Extract
#### Scrapping for Data

Now we begin scraping the setlist data!

In [5]:
url = 'https://phish.net/setlists/?year='
all_years_dates = []
all_venues = []
all_cities = []
all_states = []

# Define a mapping for known exceptions
exception_mapping = {
    "Quintana Roo": "MX",
    "Cancun": "MX",
    "London": "GB",
    "Somerset": "GB",  # Add other exceptions as needed
    "England": "GB",   # Handling England explicitly
    "United Kingdom": "GB",
}

def get_country_code(location):
    # Check if the location is in the exception mapping
    if location in exception_mapping:
        return exception_mapping[location]
    try:
        # Check if the location is a valid country name
        return pycountry.countries.lookup(location).alpha_2
    except LookupError:
        # If the country is not found, return None
        return None

def get_state_or_country(location):
    # Split the location by comma
    parts = location.split(',')
    city = parts[0].strip().title()
    if len(parts) > 1:
        state_or_country = parts[-1].strip().title()
        # Try to get country code
        country_code = get_country_code(state_or_country)
        if country_code:
            return city, country_code
        else:
            # If not a country, assume it's a state
            return ', '.join(parts[:-1]).strip().title(), state_or_country.upper()
    return city, None

# Get the current year
current_year = datetime.now().year

# Loop from 1982 to the current year
for year in range(1982, current_year + 1):
    year_url = url + str(year)
    response = requests.get(year_url)
    phish_soup = BeautifulSoup(response.text, 'html.parser')

    # Dates
    dates = phish_soup.find_all('span', class_='setlist-date')
    date_strings = [date.text[-11:] for date in dates]
    cleaned_date_strings = [datetime.strptime(date.strip(), '%m/%d/%Y').strftime('%Y-%m-%d') for date in date_strings]
    all_years_dates.extend(cleaned_date_strings)

    # Venues
    venues = phish_soup.find_all('div', class_='setlist-venue')
    venue_strings = [venue.find('span').text.strip().title().replace("'S", "'s") for venue in venues]
    all_venues.extend(venue_strings)

    # Locations/ City/ State
    locations = phish_soup.find_all('div', class_='setlist-location')
    locations = [location.text.strip() for location in locations]

    cities_states = [get_state_or_country(location) for location in locations]
    cities = [city for city, _ in cities_states]
    states_or_countries = [state_or_country for _, state_or_country in cities_states]

    all_cities.extend(cities)
    all_states.extend(states_or_countries)

phish_shows_df = pd.DataFrame({
    'Date': all_years_dates,
    'Venue': all_venues,
    'City': all_cities,
    'State': all_states
})
phish_shows_df

,Date,Venue,City,State
0,1983-12-03,"Marsh / Austin / Tupper Dormitory, University ...",Burlington,VT
1,1983-12-02,Harris-Millis Cafeteria - University Of Vermont,Burlington,VT
2,1983-10-30,Harris-Millis Cafeteria - University Of Vermont,Burlington,VT
3,1984-12-01,Nectar's,Burlington,VT
4,1984-11-03,"Slade Hall, University Of Vermont",Burlington,VT
...,...,...,...,...
2120,2024-02-24,Moon Palace,Cancun,MX
2121,2024-02-23,Moon Palace,Cancun,MX
2122,2024-02-22,Moon Palace,Cancun,MX
2123,2024-02-21,Moon Palace,Cancun,MX


In [13]:
# Check if any Country/City codes need updating
phish_shows_df[phish_shows_df['State'].str.len() > 2]

,Date,Venue,City,State


In [ ]:
# Check any other strange data
phish_shows_df[phish_shows_df['State'] == "US"]

In [10]:
phish_shows_df['State'].value_counts()

State
NY    323
VT    263
CA    158
CO    118
MA    109
PA     82
VA     56
GA     55
OH     53
NJ     51
IL     51
ME     48
NC     47
WA     44
NH     40
IN     39
TN     39
NV     38
FL     37
WI     34
CT     30
MX     30
TX     27
MI     26
OR     26
MD     26
RI     26
SC     25
MO     20
DE     20
AZ     18
AL     17
MN     16
IT     12
JP     12
LA     11
DC     11
FR     11
KS      8
DK      8
UT      7
NM      7
IA      5
GB      5
KY      5
NL      4
ES      4
BE      3
CZ      3
IE      2
AR      2
ID      2
NE      2
MT      2
WV      2
MS      2
AT      1
OK      1
US      1
Name: count, dtype: int64